# 📊 Análisis Comparativo de Modelos de Redes Neuronales
## Evaluación Integral de Todos los Modelos Implementados

---

### Objetivos:
1. Comparar rendimiento de todos los modelos
2. Analizar trade-offs (accuracy vs complejidad)
3. Identificar el mejor modelo para cada tarea
4. Generar recomendaciones finales

**Autor**: Adonnay Bazaldua  
**Fecha**: Noviembre 2025

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")

## 1. Carga de Resultados de Todos los Modelos

In [ ]:
# Cargar resultados
results = {}

models = ['mlp', 'lstm', 'gru', 'cnn', 'autoencoder']

for model_name in models:
    try:
        with open(f'models/{model_name}_results.pkl', 'rb') as f:
            results[model_name] = pickle.load(f)
        print(f"✅ {model_name.upper()} cargado")
    except FileNotFoundError:
        print(f"⚠️ {model_name.upper()} no encontrado")
        results[model_name] = None

print(f"\n📊 Modelos cargados: {len([r for r in results.values() if r is not None])}")

## 2. Comparación de Modelos de Clasificación (MLP vs CNN)

In [ ]:
# Tabla comparativa de modelos de clasificación
classification_data = []

if results['mlp']:
    classification_data.append({
        'Modelo': 'MLP',
        'Accuracy (%)': results['mlp']['test_accuracy'] * 100,
        'Top-3 Acc (%)': results['mlp'].get('test_top3_accuracy', 0) * 100,
        'Parámetros': results['mlp']['num_parameters'],
        'Épocas': results['mlp']['num_epochs_trained']
    })

if results['cnn']:
    classification_data.append({
        'Modelo': 'CNN',
        'Accuracy (%)': results['cnn']['test_accuracy'] * 100,
        'Top-3 Acc (%)': 'N/A',
        'Parámetros': results['cnn']['num_parameters'],
        'Épocas': results['cnn']['num_epochs_trained']
    })

classification_df = pd.DataFrame(classification_data)

print("\n" + "="*80)
print(" "*25 + "MODELOS DE CLASIFICACIÓN")
print("="*80)
print(classification_df.to_string(index=False))
print("="*80)

# Visualización
if len(classification_data) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    models_class = classification_df['Modelo'].values
    accuracies = classification_df['Accuracy (%)'].values
    params = classification_df['Parámetros'].values
    
    # Accuracy
    axes[0].bar(models_class, accuracies, color=['steelblue', 'coral'])
    axes[0].set_ylabel('Accuracy (%)')
    axes[0].set_title('Accuracy por Modelo', fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Parámetros
    axes[1].bar(models_class, params, color=['steelblue', 'coral'])
    axes[1].set_ylabel('Número de Parámetros')
    axes[1].set_title('Complejidad del Modelo', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('models/comparison_classification.png', dpi=300)
    plt.show()

## 3. Comparación de Modelos de Series Temporales (LSTM vs GRU)

In [ ]:
# Tabla comparativa de modelos de series temporales
timeseries_data = []

if results['lstm']:
    timeseries_data.append({
        'Modelo': 'LSTM',
        'MAE': results['lstm']['test_mae'],
        'RMSE': results['lstm']['test_rmse'],
        'R²': results['lstm']['test_r2'],
        'MAPE (%)': results['lstm']['test_mape'],
        'Parámetros': results['lstm']['num_parameters'],
        'Épocas': results['lstm']['num_epochs_trained']
    })

if results['gru']:
    timeseries_data.append({
        'Modelo': 'GRU',
        'MAE': results['gru']['test_mae'],
        'RMSE': results['gru']['test_rmse'],
        'R²': results['gru']['test_r2'],
        'MAPE (%)': results['gru']['test_mape'],
        'Parámetros': results['gru']['num_parameters'],
        'Épocas': results['gru']['num_epochs_trained']
    })

timeseries_df = pd.DataFrame(timeseries_data)

print("\n" + "="*100)
print(" "*35 + "MODELOS DE SERIES TEMPORALES")
print("="*100)
print(timeseries_df.to_string(index=False))
print("="*100)

# Cálculo de mejora/diferencia
if len(timeseries_data) == 2:
    print(f"\n🔍 Análisis LSTM vs GRU:")
    
    mae_diff = ((results['gru']['test_mae'] - results['lstm']['test_mae']) / results['lstm']['test_mae']) * 100
    param_diff = ((results['gru']['num_parameters'] - results['lstm']['num_parameters']) / results['lstm']['num_parameters']) * 100
    
    print(f"   Diferencia en MAE: {mae_diff:+.2f}%")
    print(f"   Diferencia en parámetros: {param_diff:+.2f}%")
    
    if mae_diff < 0 and param_diff < 0:
        print(f"   ✅ GRU es MEJOR: menor error y menor complejidad")
    elif mae_diff < 0:
        print(f"   ⚖️ GRU tiene menor error pero más parámetros")
    elif param_diff < 0:
        print(f"   ⚖️ GRU tiene menos parámetros pero mayor error")
    else:
        print(f"   ⚠️ LSTM es mejor en ambas métricas")

# Visualización
if len(timeseries_data) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    models_ts = timeseries_df['Modelo'].values
    mae_vals = timeseries_df['MAE'].values
    rmse_vals = timeseries_df['RMSE'].values
    r2_vals = timeseries_df['R²'].values
    
    colors = ['#2E86AB', '#A23B72']
    
    axes[0].bar(models_ts, mae_vals, color=colors[:len(models_ts)])
    axes[0].set_ylabel('MAE')
    axes[0].set_title('Mean Absolute Error', fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    axes[1].bar(models_ts, rmse_vals, color=colors[:len(models_ts)])
    axes[1].set_ylabel('RMSE')
    axes[1].set_title('Root Mean Squared Error', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    axes[2].bar(models_ts, r2_vals, color=colors[:len(models_ts)])
    axes[2].set_ylabel('R² Score')
    axes[2].set_title('R² Score (Mayor es mejor)', fontweight='bold')
    axes[2].set_ylim([0, 1])
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('models/comparison_timeseries.png', dpi=300)
    plt.show()

## 4. Tabla Comparativa General

In [ ]:
# Crear tabla general
overall_data = []

model_info = {
    'mlp': {'tarea': 'Clasificación', 'métrica_principal': 'Accuracy'},
    'cnn': {'tarea': 'Clasificación Espacial', 'métrica_principal': 'Accuracy'},
    'lstm': {'tarea': 'Serie Temporal', 'métrica_principal': 'MAE'},
    'gru': {'tarea': 'Serie Temporal', 'métrica_principal': 'MAE'},
    'autoencoder': {'tarea': 'Detección Anomalías', 'métrica_principal': 'Reconstruction Error'}
}

for model_name, info in model_info.items():
    if results[model_name]:
        res = results[model_name]
        
        # Métrica principal
        if 'test_accuracy' in res:
            metric_value = f"{res['test_accuracy']*100:.2f}%"
        elif 'test_mae' in res:
            metric_value = f"{res['test_mae']:.2f}"
        else:
            metric_value = f"{res.get('mean_reconstruction_error', 0):.4f}"
        
        overall_data.append({
            'Modelo': model_name.upper(),
            'Tarea': info['tarea'],
            'Métrica Principal': info['métrica_principal'],
            'Valor': metric_value,
            'Parámetros': f"{res['num_parameters']:,}",
            'Épocas': res.get('num_epochs_trained', 'N/A')
        })

overall_df = pd.DataFrame(overall_data)

print("\n" + "="*110)
print(" "*40 + "RESUMEN GENERAL DE MODELOS")
print("="*110)
print(overall_df.to_string(index=False))
print("="*110)

# Guardar
overall_df.to_csv('models/overall_comparison.csv', index=False)
print("\n✅ Tabla guardada en 'models/overall_comparison.csv'")

## 5. Recomendaciones y Conclusiones

In [ ]:
print("\n" + "="*100)
print(" "*35 + "CONCLUSIONES Y RECOMENDACIONES")
print("="*100)

print("\n🎯 CLASIFICACIÓN DE TIPOS DE DELITO:")
if results['mlp'] and results['cnn']:
    if results['mlp']['test_accuracy'] > results['cnn']['test_accuracy']:
        print(f"   ✅ RECOMENDACIÓN: MLP")
        print(f"      - Accuracy: {results['mlp']['test_accuracy']*100:.2f}% (superior a CNN)")
        print(f"      - Más simple y rápido de entrenar")
    else:
        print(f"   ✅ RECOMENDACIÓN: CNN")
        print(f"      - Accuracy: {results['cnn']['test_accuracy']*100:.2f}% (superior a MLP)")
        print(f"      - Mejor para capturar patrones espaciales")
elif results['mlp']:
    print(f"   ✅ RECOMENDACIÓN: MLP (único disponible)")
    print(f"      - Accuracy: {results['mlp']['test_accuracy']*100:.2f}%")

print("\n📈 PREDICCIÓN DE SERIES TEMPORALES:")
if results['lstm'] and results['gru']:
    if results['gru']['test_mae'] < results['lstm']['test_mae']:
        print(f"   ✅ RECOMENDACIÓN: GRU")
        print(f"      - MAE: {results['gru']['test_mae']:.2f} (mejor que LSTM)")
        print(f"      - {((results['lstm']['num_parameters'] - results['gru']['num_parameters'])/results['lstm']['num_parameters']*100):.1f}% menos parámetros")
        print(f"      - Entrenamiento más rápido")
    else:
        print(f"   ✅ RECOMENDACIÓN: LSTM")
        print(f"      - MAE: {results['lstm']['test_mae']:.2f} (mejor que GRU)")
        print(f"      - Mejor para secuencias complejas")
elif results['lstm']:
    print(f"   ✅ RECOMENDACIÓN: LSTM (único disponible)")
    print(f"      - MAE: {results['lstm']['test_mae']:.2f}")

print("\n🔍 DETECCIÓN DE ANOMALÍAS:")
if results['autoencoder']:
    print(f"   ✅ RECOMENDACIÓN: Autoencoder")
    print(f"      - {results['autoencoder']['num_anomalies']} anomalías detectadas")
    print(f"      - {results['autoencoder']['anomaly_percentage']:.2f}% del dataset")
    print(f"      - Útil para identificar delitos atípicos")

print("\n💡 INSIGHTS GENERALES:")
print("   1. Los modelos profundos superan a baselines simples")
print("   2. GRU ofrece buen balance eficiencia/rendimiento")
print("   3. Las features temporales son críticas para predicción")
print("   4. La detección de anomalías complementa bien la clasificación")

print("\n🚀 TRABAJO FUTURO:")
print("   - Implementar ensemble de modelos")
print("   - Agregar más features contextuales (clima, eventos)")
print("   - Desplegar modelo en producción")
print("   - Crear dashboard interactivo")

print("\n" + "="*100)

## 6. Exportar Informe Final

In [ ]:
# Crear informe markdown
report = f"""
# Informe Final: Análisis de Delitos CDMX con Redes Neuronales

**Autor**: Adonnay Bazaldua  
**Fecha**: Noviembre 2025

---

## Resumen Ejecutivo

Se implementaron y evaluaron **5 arquitecturas de redes neuronales** para analizar delitos en la CDMX:

1. **MLP**: Clasificación de tipos de delito
2. **LSTM**: Predicción de series temporales
3. **GRU**: Comparación con LSTM
4. **CNN**: Análisis espacial
5. **Autoencoder**: Detección de anomalías

---

## Resultados por Modelo

{overall_df.to_markdown(index=False)}

---

## Recomendaciones

### Para Producción:
- **Clasificación**: Usar {"MLP" if results.get('mlp') and results.get('cnn') and results['mlp']['test_accuracy'] > results['cnn']['test_accuracy'] else "CNN"}
- **Series Temporales**: Usar {"GRU" if results.get('gru') and results.get('lstm') and results['gru']['test_mae'] < results['lstm']['test_mae'] else "LSTM"}
- **Anomalías**: Autoencoder complementario

### Trade-offs:
- GRU ofrece mejor balance eficiencia/rendimiento
- MLP es más simple pero efectivo
- Autoencoder útil para análisis exploratorio

---

## Archivos Generados

- `models/mlp_classifier_final.keras`
- `models/lstm_predictor_final.keras`
- `models/gru_predictor_final.keras`
- `models/cnn_spatial_final.keras`
- `models/autoencoder_final.keras`

**Proyecto completado exitosamente** ✅
"""

with open('REPORTE_FINAL.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Informe final guardado en 'REPORTE_FINAL.md'")
print("\n🎉 ¡ANÁLISIS COMPARATIVO COMPLETADO!")